In [1]:
# change tabula to tabula_middle_padding to test middle padding method
from tabula import Tabula 
import pandas as pd
import os
import sys

sys.path.append("/tf/notebooks/Tabula-main")

os.environ["CUDA_VISIBLE_DEVICES"]="1"



# 设置环境变量

import os
%env NCCL_P2P_DISABLE=1
%env NCCL_DEBUG=INFO

%env NCCL_IB_DISABLE=1
# 获取NCCL_IB_DISABLE的环境变量值
nccl_ib_disable = os.getenv('NCCL_IB_DISABLE')

# 打印环境变量的值
print(f"The current value for NCCL_IB_DISABLE is: {nccl_ib_disable}")

/opt/conda/envs/tabula/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


env: NCCL_P2P_DISABLE=1
env: NCCL_DEBUG=INFO
env: NCCL_IB_DISABLE=1
The current value for NCCL_IB_DISABLE is: 1


In [2]:
data = pd.read_csv("Real_Datasets/Insurance_compressed.csv")

In [3]:
categorical_columns = ["sex", "children", "sm", "region"]
model = Tabula(llm='distilgpt2', experiment_dir = "insurance_training", batch_size=32, epochs=400, categorical_columns = categorical_columns)

In [4]:
# Comment this block out to test tabula starting from randomly initialized model.
# Comment this block out when uses tabula_middle_padding
import torch
model.model.load_state_dict(torch.load("pretrained-model/adult_king_insurance_intrusion_covtype_pretrained.pt"), strict=False)

/tmp/ipykernel_86431/3909679578.py:4: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load("pretrained-model/adult_king_insurance_intrusion_c

<All keys matched successfully>

In [5]:
model.fit(data)

You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
500,1.364700
1000,0.906100
1500,0.763700
2000,0.706900
2500,0.676900
3000,0.661000
3500,0.647900
4000,0.635900
4500,0.627700
5000,0.620900


In [6]:
import torch
torch.save(model.model.state_dict(), "insurance_training/model_400epoch.pt")

In [7]:
synthetic_data = model.sample(n_samples=1338)
synthetic_data.to_csv("insurance_400epoch.csv", index=False)

  0%|          | 0/1338 [00:00<?, ?it/s]The attention mask is not set and cannot be inferred from input because pad token is same as eos token.As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
1346it [00:03, 394.19it/s]                          
/tf/notebooks/Tabula-main/tabula/tabula.py:103: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data[self.label_encoder_list[i]['column']] = pd.to_numeric(data[self.label_encoder_list[i]['column']], errors='coerce')
